<a href="https://colab.research.google.com/github/sid8123/quantum_optimization/blob/master/maxcut_QAOA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install qiskit


In [27]:
from collections import OrderedDict
import numpy as np
from qiskit.quantum_info import Pauli

###### This is the function I couldn't find updated version ########## 
# from qiskit_aqua import Operator
### also change qiskit_aqua to qiskit.aqua #####
############################################
from qiskit.optimization.applications.ising import max_cut

from qiskit.aqua.algorithms import QAOA
from qiskit.aqua.components.optimizers import *
from qiskit.aqua import QuantumInstance
import networkx as nx
# from qiskit.aqua import get_aer_backend
from qiskit import Aer, execute


def sample_most_likely(state_vector):
    if isinstance(state_vector, dict) or isinstance(state_vector, OrderedDict):
        binary_string = sorted(state_vector.items(), key=lambda kv: kv[1])[-1][0]
        x = np.asarray([int(y) for y in reversed(list(binary_string))])
        return x
    else:
        n = int(np.log2(state_vector.shape[0]))
        k = np.argmax(np.abs(state_vector))
        x = np.zeros(n)
        for i in range(n):
            x[i] = k % 2
            k >>= 1
        return x
# def get_qubitops(input):
#     w = np.array(input.tolist())
#     num_nodes = len(w)
#     pauli_list = []
#     shift = 0
#     for i in range(num_nodes):
#         for j in range(i+1, num_nodes):
#             if w[i, j] != 0:
#                 xp = np.zeros(num_nodes, dtype=np.bool)
#                 zp = np.zeros(num_nodes, dtype=np.bool)
#                 zp[i] = True
#                 zp[j] = True
#                 pauli_list.append([w[i, j], Pauli(zp, xp)])
#                 shift += 1
#     for i in range(num_nodes):
#         degree = np.sum(w[i, :])
#         xp = np.zeros(num_nodes, dtype=np.bool)
#         zp = np.zeros(num_nodes, dtype=np.bool)
#         zp[i] = True
#         pauli_list.append([w[i, i], Pauli(zp, xp)])
#     return Operator(paulis=pauli_list)

#'ibmqx4'
#'ibmq_16_melbourne'
# from qiskit import IBMQ
# IBMQ.load_accounts()

def solve_ibmqx_ising_qubo(G, matrix_func, optimizer, p):
        backend = Aer.get_backend('qasm_simulator')
        w = matrix_func(G)
        # ops = get_qubitops(w)
        ops, offset = max_cut.get_operator(w)
        qaoa = QAOA(ops, optimizer, p, quantum_instance=backend)
        quantum_instance = QuantumInstance(backend)
        result = qaoa.run(quantum_instance)
        x = sample_most_likely(result['eigvecs'][0])
        return x
# def solve_ibmqx_ising_qubo_nisq_melbourne(G, matrix_func, optimizer, p):
#         backend = IBMQ.get_backend('ibmq_16_melbourne')
#         w = matrix_func(G)
#         ops = get_qubitops(w)
#         qaoa = QAOA(ops, optimizer, p, operator_mode='paulis')
#         quantum_instance = QuantumInstance(backend)
#         result = qaoa.run(quantum_instance)
#         x = sample_most_likely(result['eigvecs'][0])
#         return x
# def solve_ibmqx_ising_qubo_nisq_ibmqx4(G, matrix_func, optimizer, p):
#         backend = IBMQ.get_backend('ibmqx4')
#         w = matrix_func(G)
#         ops = get_qubitops(w)
#         qaoa = QAOA(ops, optimizer, p, operator_mode='paulis')
#         quantum_instance = QuantumInstance(backend)
#         result = qaoa.run(quantum_instance)
#         x = sample_most_likely(result['eigvecs'][0])
#         return x


In [28]:
import networkx as nx

def ibmqx4():
        G = nx.Graph()
        G.add_edge(0, 1)
        G.add_edge(2, 1)
        G.add_edge(2, 0)
        G.add_edge(3, 2)
        G.add_edge(2, 4)
        G.add_edge(3, 4)
        return G
def melbourne():
        G = nx.Graph()
        G.add_edge(0, 1)
        G.add_edge(1, 2)
        G.add_edge(13, 1)
        G.add_edge(13, 12)
        G.add_edge(12, 2)
        G.add_edge(2, 3)
        G.add_edge(11, 12)
        G.add_edge(11, 3)
        G.add_edge(4, 3)
        G.add_edge(11, 10)
        G.add_edge(4, 10)
        G.add_edge(5, 4)
        G.add_edge(9, 10)
        G.add_edge(5, 9)
        G.add_edge(5, 6)
        G.add_edge(9, 8)
        G.add_edge(6, 8)
        G.add_edge(7, 8)
        return G
def agave():
        G = nx.Graph()
        G.add_edge(0, 1)
        G.add_edge(1, 2)
        G.add_edge(2, 3)
        G.add_edge(3, 4)
        G.add_edge(4, 5)
        G.add_edge(5, 6)
        G.add_edge(6, 7)
        G.add_edge(7, 0)
        return G
def acorn():
        G = nx.Graph()
        G.add_edge(0, 5)
        G.add_edge(0, 6)
        G.add_edge(5, 10)
        G.add_edge(10, 15)
        G.add_edge(10, 16)
        G.add_edge(16, 11)
        G.add_edge(11, 17)
        G.add_edge(1, 6)
        G.add_edge(1, 7)
        G.add_edge(2, 7)
        G.add_edge(17, 12)
        G.add_edge(12, 18)
        G.add_edge(18, 13)
        G.add_edge(13, 19)
        G.add_edge(19, 14)
        G.add_edge(9, 14)
        G.add_edge(4, 9)
        G.add_edge(3, 9)
        G.add_edge(3, 8)
        G.add_edge(2, 8)
        G.add_edge(6, 11)
        G.add_edge(7, 12)
        G.add_edge(8, 13)
        return G
def aspen():
        G = nx.Graph()
        G.add_edge(6, 5)
        G.add_edge(5, 4)
        G.add_edge(4, 3)
        G.add_edge(3, 2)
        G.add_edge(2, 1)
        G.add_edge(1, 0)
        G.add_edge(0, 7)
        G.add_edge(7, 6)
        G.add_edge(15, 16)
        G.add_edge(16, 17)
        G.add_edge(17, 10)
        G.add_edge(10, 11)
        G.add_edge(11, 12)
        G.add_edge(13, 12)
        G.add_edge(13, 14)
        G.add_edge(15, 14)
        G.add_edge(2, 15)
        G.add_edge(1, 16)
        return G

In [29]:
"""
QUBO Generators from NetworkX Graph Objects

Max Cut QUBO formulation from 
https://www.sciencedirect.com/science/article/pii/S0166218X01003419 

Minimum Vertex Cover Ising formluations from 
https://arxiv.org/abs/1302.5843

Maximum clique and maximum indpendent set from
https://arxiv.org/pdf/1801.08649.pdf

"""
import numpy as np
import networkx as nx

def maximum_clique_qubo_rigetti(G):
        lin = []
        quad = {}
        GC = nx.algorithms.operators.unary.complement(G)
        for i in list(G.nodes()):
                lin.append(-1)
        for a in list(GC.edges()):
                quad[a] = 2
        return lin, quad
def maximum_clique_qubo_dwave(G):
        Q = {}
        GC = nx.algorithms.operators.unary.complement(G)
        for i in list(G.nodes()):
                Q[(i, i)] = -1
        for a in list(GC.edges()):
                Q[a] = 2
        return Q

def maximum_cut_qubo_rigetti(G):
        quad = {}
        lin = []
        for a in list(G.edges()):
                quad[a] = 2
        for i in list(G.nodes()):
                lin.append(-1*G.degree(i))
        return lin, quad
def maximum_cut_qubo_dwave(G):
        Q = {}
        for a in list(G.edges()):
                Q[a] = 2
        for i in list(G.nodes()):
                Q[(i, i)] = -1*G.degree(i)
        return Q

def minimum_vertex_cover_qubo_rigetti(G):
	J = {}
	h = []
	for a in list(G.edges()):
		J[a] = 2
	for i in list(G.nodes()):
		h.append((-2*G.degree(i))+1)
	return h, J
def minimum_vertex_cover_qubo_dwave(G):
	Q = {}
	for a in list(G.edges()):
		Q[a] = 2
	for i in list(G.nodes()):
		Q[(i, i)] = ((-1*G.degree(i))-1)
	return Q
def max_cut_qubo_matrix_ibmqx(G):
        qubo = maximum_cut_qubo_dwave(G)
        data = np.zeros((len(G), len(G)))
        for a in qubo:
                data[a[0], a[1]] = qubo[a]
                data[a[1], a[0]] = qubo[a]
        return data

def minimum_vertex_cover_qubo_matrix_ibmqx(G):
        h, J = minimum_vertex_cover_qubo_rigetti(G)
        data = np.zeros((len(G), len(G)))
        for a in J:
                data[a[0], a[1]] = J[a]
                data[a[1], a[0]] = J[a]
        count = -1
        for i in h:
                count += 1
                data[count, count] = i
        return data

def max_clique_qubo_matrix_ibmqx(G):
        qubo = maximum_clique_qubo_dwave(G)
        data = np.zeros((len(G), len(G)))
        for a in qubo:
                data[a[0], a[1]] = qubo[a]
                data[a[1], a[0]] = qubo[a]
        return data

In [31]:
import networkx as nx
import itertools

def maximum_clique(G):
	return nx.algorithms.clique.graph_clique_number(G)
def maximum_independent_set(G):
	GC = nx.algorithms.operators.unary.complement(G)
	return nx.algorithms.clique.graph_clique_number(GC)
def minimum_vertex_cover(G):
        GC = nx.algorithms.operators.unary.complement(G)
        return len(G) - nx.algorithms.clique.graph_clique_number(GC)
def is_clique(G):
	n = len(G)
	m = len(G.edges())
	if m == ((n*(n-1))/2):
		return True
	else:
		return False
def max_cut_value(vector, G):
	count = -1
	subg1 = []
	subg2 = []
	for a in vector:
		count += 1
		if a == 1:
			subg1.append(count)
		if a != 1:
			subg2.append(count)
	H1 = G.subgraph(subg1)
	H2 = G.subgraph(subg2)
	m1 = len(H1.edges())
	m2 = len(H2.edges())
	val = len(G.edges())-(m1+m2)
	return val
def is_independent_set(vector, G):
        count = -1
        subg1 = []
        for a in vector:
                count += 1
                if a == 1:
                        subg1.append(count)
        H = G.subgraph(subg1)
        HC = nx.algorithms.operators.unary.complement(H)
        if is_clique(HC) == True:
                return True
        else:
                return False
def list_difference(list1, list2):
	out = []
	for a in list1:
		if a not in list2:
			out.append(a)
	return out
def is_vertex_cover(vector, G):
        count = -1
        subg1 = []
        for a in vector:
                count += 1
                if a == 1:
                        subg1.append(count)
        list1 = list(G.nodes())
        mis_subg = list_difference(list1, subg1)
        H = G.subgraph(mis_subg)
        HC = nx.algorithms.operators.unary.complement(H)
        if is_clique(HC) == True:
                return True
        else:
                return False
def subg_is_clique(list, G):
	count = -1
	mc = []
	for i in list:
		count += 1
		if i == 1:
			mc.append(count)
	H = G.subgraph(mc)
	if is_clique(H) == True:
		return True
	else:
		return False

In [ ]:
import os
import sys
d = os.path.dirname(os.getcwd())+'/utils'
sys.path.append(d)
#from graphs import *
#from ibmqx_ising_qubo_qaoa import *
#from qubo_ising_generators import *
#from classical_solvers import *
import networkx as nx
import numpy as np
import time
import warnings
warnings.filterwarnings("ignore")

def ibqmx_qaoa(G, optimizer, func, r):
        app_ratio = 0
        qubo_main = []
        counter = 0
        for avg in range(0, 100):
            counter += 1
            result_out = solve_ibmqx_ising_qubo(G, func, optimizer, r)
            result2 = []
            for i in result_out:
                  if i == 0:
                        result2.append(1)
                  if i == 1:
                        result2.append(0)
            x = max_cut_value(result2, G)
            qubo_main.append(x)
            app_ratio += x
            print(float(app_ratio)/float(counter))

print('maxcut')
out = []
for r in range(1, 20):
    G = nx.gnp_random_graph(7, 0.5, 101)
    res = ibqmx_qaoa(G, SLSQP(), max_cut_qubo_matrix_ibmqx, r)
    out.append(res)
    print(out)

maxcut
7.0
6.0
6.666666666666667
5.75
5.6
5.666666666666667
5.857142857142857
5.75
5.888888888888889
5.8
5.7272727272727275
5.916666666666667
5.923076923076923
5.857142857142857
5.866666666666666
5.8125
5.882352941176471
5.888888888888889
6.0
6.0
6.095238095238095
6.090909090909091
6.130434782608695
6.166666666666667
6.12
6.115384615384615
6.222222222222222
6.178571428571429
6.206896551724138
6.066666666666666
6.064516129032258
6.125
6.181818181818182
6.176470588235294
6.228571428571429
6.138888888888889
6.135135135135135
6.184210526315789
6.205128205128205
6.275
6.219512195121951
6.238095238095238
6.162790697674419
6.25
6.266666666666667
6.304347826086956
6.276595744680851
6.145833333333333
6.204081632653061
6.24
6.294117647058823
6.288461538461538
6.169811320754717
6.203703703703703
6.236363636363636
6.25
6.2105263157894735
6.224137931034483
6.237288135593221
6.25
6.278688524590164
6.258064516129032
6.285714285714286
6.296875
6.323076923076923
6.363636363636363
6.268656716417911
6.26